This notebook is used to merge crawled data from different sources, preprocess them and write to to DB before indexing process

In [8]:
# !pip install polyglot
# !pip install polyglot
# !pip install PyICU
# !pip install pycld2
# !pip install langdetect
# !pip install swifter

In [9]:
import duckdb
import pandas as pd
import swifter
from langdetect import detect
from polyglot.detect import Detector

In [13]:
conn_existing = duckdb.connect('crawlerDb.duckdb')
conn_new = duckdb.connect('crawler_v4.db')

In [14]:
df_existing = conn_existing.execute("select * from urlsDB").df()
df_new = conn_new.execute("select * from urlsDB").df()

In [17]:
# to filter out duplicate URLs
def normalise_url(url):
    if url.startswith("https://"):
        return url[8:]
    elif url.startswith("http://"):
        return url[7:]
    # remove query parameters
    if '?' in url:
        url = url.split('?')[0]
    # remove trailing slashes
    if url.endswith('/'):
        url = url[:-1]
    return url

In [18]:
df_existing['normalised_url'] = df_existing['url'].swifter.apply(normalise_url)
df_new['normalised_url'] = df_new['url'].swifter.apply(normalise_url)

Pandas Apply: 100%|██████████| 144224/144224 [00:00<00:00, 1938831.48it/s]


In [21]:
# remove duplicates
print(f'len(df_new): {len(df_new)}')
df_new = df_new[~df_new['normalised_url'].isin(df_existing['normalised_url'])]
print(f'len(df_new) after removing existing: {len(df_new)}')
# remove duplicates within df_new
df_new = df_new.drop_duplicates(subset=['normalised_url'])
print(f'len(df_new) after removing duplicates: {len(df_new)}')

len(df_new): 144224
len(df_new) after removing existing: 115494
len(df_new) after removing duplicates: 115441


In [ ]:
df_new.drop(columns=['normalised_url'], inplace=True)
df_existing.drop(columns=['normalised_url'], inplace=True)

## Language Deetection:
Detecting if a text is in English isn't quite easy. Some of the existing python packages struggle with either very short or long texts. To be lenient, we try detetcting the english language from libraries langdetect and polygot. If either of them says the language of the text is english with a confidence of atleast 15% we accept, else we reject it

In [ ]:
def is_english_langdetect(text):
    if len(text) > 2:
        try:
            return detect(text) == 'en'
        except:
            print(f"Error detecting language for text: {text[:100]}...")  
            return False
    return False

def is_english_polyglot(text):
    try:
        detector = Detector(text)
        for language in detector.languages:
            if language.code == 'en' and language.confidence > 0.15:
                return True
        return False
    except Exception as e:
        print(f"Error detecting language for text: {text[:100]}... Error: {e}") 
        return False


In [25]:
df_new['is_english_langdetect'] = df_new['text'].swifter.apply(is_english_langdetect)
df_new['is_english_polyglot'] = df_new['text'].swifter.apply(is_english_polyglot)

Pandas Apply:  91%|█████████ | 104878/115441 [10:25<00:57, 184.10it/s]

Error detecting language for text: "constituency" သို့ ချိတ်ဆက်ထားသော စာမျက်နှာများ - ဝစ်ရှင်နရီ မာတိကာသို့ ခုန်သွားရန် ပင်မ မီနူး ပင်မ...
Error detecting language for text: "constituent" သို့ ချိတ်ဆက်ထားသော စာမျက်နှာများ - ဝစ်ရှင်နရီ မာတိကာသို့ ခုန်သွားရန် ပင်မ မီနူး ပင်မ ...


Pandas Apply:  94%|█████████▎| 108222/115441 [10:42<00:41, 173.89it/s]

Error detecting language for text: "constituency" သို့ ချိတ်ဆက်ထားသော စာမျက်နှာများ - ဝစ်ရှင်နရီ ပင်မ ကျပန်း အကောင့်ဝင်ရန် အပြင်အဆင်မျာ...
Error detecting language for text: "constituent" သို့ ချိတ်ဆက်ထားသော စာမျက်နှာများ - ဝစ်ရှင်နရီ မာတိကာသို့ ခုန်သွားရန် ပင်မ မီနူး ပင်မ ...


Pandas Apply:  94%|█████████▍| 108260/115441 [10:42<00:41, 174.71it/s]

...

Pandas Apply:  99%|█████████▊| 113950/115441 [00:41<00:00, 3126.07it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Pandas Apply:  99%|█████████▉| 114430/115441 [00:41<00:00, 3520.73it/s]

Error detecting language for text: Where to study in Paris and in the Île-de-France - La Chancellerie des Universités de Paris Intranet... Error: Try passing a longer snippet of text
Error detecting language for text: Organisation and Missions - La Chancellerie des Universités de Paris Intranet FR EN La Chancellerie ... Error: Try passing a longer snippet of text
Error detecting language for text: The Sorbonne in the 19th century - La Chancellerie des Universités de Paris Intranet FR EN La Chance... Error: Try passing a longer snippet of text
Error detecting language for text: Villa Finaly - La Chancellerie des Universités de Paris Intranet FR EN La Chancellerie des Universit... Error: Try passing a longer snippet of text
Error detecting language for text: The Sorbonne in the 20th century - La Chancellerie des Universités de Paris Intranet FR EN La Chance... Error: Try passing a longer snippet of text
Error detecting language for text: ï»¿ Home function checkClickAndCloseNav() { if ($(

Pandas Apply: 100%|█████████▉| 115178/115441 [00:41<00:00, 3444.83it/s]

Error detecting language for text: Category:Privacy policy - Wikimedia Foundation Governance Wiki Home Random Nearby Log in Settings Do... Error: Try passing a longer snippet of text
Error detecting language for text: Category:Policies - Wikimedia Foundation Governance Wiki Home Random Nearby Log in Settings Donate N... Error: Try passing a longer snippet of text
Error detecting language for text: Migration consultants - Police and Border Guard Board Move to content Accessibility î  Emergency 11... Error: input contains invalid UTF-8 around byte 87 (of 1972)
Error detecting language for text: US2352523A - Apparatus and method for artificially inducing breathing - Google Patents US2352523A - ... Error: input contains invalid UTF-8 around byte 6539 (of 60131)
Error detecting language for text: US2468741A - Breathing apparatus - Google Patents US2468741A - Breathing apparatus - Google Patents ... Error: input contains invalid UTF-8 around byte 1472 (of 93111)
Error detecting language for

Pandas Apply: 100%|██████████| 115441/115441 [00:41<00:00, 2779.65it/s]


In [26]:
df_new.is_english_langdetect.value_counts(), df_new.is_english_polyglot.value_counts()

(is_english_langdetect
 False    77467
 True     37974
 Name: count, dtype: int64,
 is_english_polyglot
 True     60373
 False    55068
 Name: count, dtype: int64)

In [ ]:
df_new_english = df_new[(df_new.is_english_langdetect == True) | (df_new.is_english_polyglot == True)]


In [ ]:
df_english = df_new_english.drop(columns=['is_english_langdetect', 'is_english_polyglot'])
df_english = df_english.drop_duplicates(subset=['url'])


In [44]:
max_id = df_existing['id'].max()
df_english['id'] = range(max_id + 1, max_id + 1 + len(df_english))


/var/folders/qn/xqf2q5g92lgd4j6z0bq50vyh0000gp/T/ipykernel_11140/840872592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['id'] = range(max_id + 1, max_id + 1 + len(df_english))


In [ ]:
# change lastFetch to unix timestamp
df_english['lastFetch'] = pd.to_datetime(df_english['lastFetch']).astype('int64') 
# rename columns
df_english = df_english.rename(columns={
    'parentUrl': 'incoming',
}, inplace=True)
# remove columns that are not in df_existing
df_english = df_english[df_existing.columns]
df_english

,id,url,title,text,lastFetch,incoming,domainLinkingDepth,linkingDepth,tueEngScore
4,66652,https://www.cmfi.uni-tuebingen.de/en/news-even...,CMFI | How gut bacteria evade the immune syste...,CMFI | How gut bacteria evade the immune syste...,1752614458834481,None,0,0,0.433333
13,66653,https://www.cmfi.uni-tuebingen.de/en/team/exec...,Prof. Dr. Andreas Peschel | Cluster of Excelle...,Prof. Dr. Andreas Peschel | Cluster of Excelle...,1752614462010076,None,0,0,0.437083
19,66654,https://www.cmfi.uni-tuebingen.de/en/news-even...,Christoph Ratzke | Stopping Microbial Invaders...,Christoph Ratzke | Stopping Microbial Invaders...,1752614464898478,None,0,0,0.454836
29,66655,https://www.cmfi.uni-tuebingen.de/en/news-even...,Ruth Ley receives Advanced Grant from the Euro...,Ruth Ley receives Advanced Grant from the Euro...,1752614472052074,None,0,0,0.452654
36,66656,https://www.geschichtswerkstatt-tuebingen.de/e...,Jüdische Spuren in Tübingen | Geschichtswerkst...,Jüdische Spuren in Tübingen | Geschichtswerkst...,1752614476950918,None,0,0,0.500000
...,...,...,...,...,...,...,...,...,...
144219,128842,https://www.staatsoper-berlin.de/en/veranstalt...,Guided Tour | Staatsoper Berlin,Guided Tour | Staatsoper Berlin Unsere Cookies...,1752744993217305,https://www.staatsoper-berlin.de/en/?source=bde,4,4,0.076563
144220,128843,https://www.staatsoper-berlin.de/en/veranstalt...,Cassandra | Staatsoper Berlin,Cassandra | Staatsoper Berlin Unsere Cookies f...,1752744997401376,https://www.staatsoper-berlin.de/en/?source=bde,4,4,0.069286
144221,128844,https://www.kau.se/en/library,Library | Karlstad University,Library | Karlstad University Skip to main con...,1752745003871211,https://www.kau.se/en/employees/jonathan-stran...,4,4,0.082857
144222,128845,https://www.kau.se/en/about-university,About the university | Karlstad University,About the university | Karlstad University Ski...,1752745013204479,https://www.kau.se/en/employees/jonathan-stran...,4,4,0.077500


In [43]:
concat_df = pd.concat([df_existing, df_english], ignore_index=True)


In [41]:
# delete the old table and create a new one
conn_existing.execute("DROP TABLE IF EXISTS urlsDB").df()

,Success


In [42]:
conn_existing.execute("CREATE TABLE urlsDB AS SELECT * FROM concat_df").df()

,Count
0,128846


In [47]:
conn_existing.close()
conn_new.close()